# FrontDash API Interactive Tests

This notebook provides one interactive test per cell to exercise the backend API endpoints used in the presentation.

Prerequisites:
- Start the backend (Docker Compose or run the Spring Boot app) so the API is available at http://localhost:8080.
- Install Python and the `requests` package if you plan to run this notebook locally: `pip install requests`.

How to use:
- Run cells one-by-one. Each cell prints the HTTP status and response body.
- The notebook stores created IDs in Python variables so subsequent cells can reference them.
- There's also an interactive runner cell near the end that will run each test in sequence and pause between them (useful for demos).

In [1]:
# Imports and basic config
import requests
import json
from datetime import datetime, timedelta

BASE = 'http://localhost:8080/api'  # change if your backend uses a different host/port

def print_resp(r):
    """Print response status and body in a readable format without blocking."""
    try:
        body = r.json()
    except Exception:
        body = r.text
    print(f"STATUS: {r.status_code}")
    # Print JSON nicely when possible, otherwise print raw text
    if isinstance(body, (dict, list)):
        print('BODY:\n' + json.dumps(body, indent=2))
    else:
        print('BODY:\n' + str(body))

# Variables that tests will populate
created_order_id = None
created_staff_username = None
created_driver_id = None
created_restaurant_id = None

## 1) Staff successful login

In [2]:
# Replace credentials with ones from your seeded data (see data.sql)
payload = {
    'username': 'staff_mike',
    'password': 'staff123'
}
try:
    r = requests.post(f'{BASE}/staff/login', json=payload, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 401
BODY:



## 2) Staff invalid login

In [3]:
payload = { 'username': 'staff_mike', 'password': 'wrongpassword' }
try:
    r = requests.post(f'{BASE}/staff/login', json=payload, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 401
BODY:



## 3) Create a new order

In [4]:
order_payload = {
  'restaurantId': 1,
  'customerName': 'Alice Presentation',
  'customerPhone': '512-999-0000',
  'address': {
     'streetAddress': '123 Maple St',
     'city': 'Austin',
     'state': 'TX',
     'zipCode': '73301'
  },
  'items': [ { 'menuItemId': 1, 'quantity': 2 } ],
  'tips': 3.00,
  'subtotal': 22.48
}
try:
    r = requests.post(f'{BASE}/orders', json=order_payload, timeout=10)
    print_resp(r)
    if r.status_code in (200,201):
        try:
            created_order = r.json()
            created_order_id = created_order.get('orderId') or created_order.get('order_id')
            print('created_order_id ->', created_order_id)
        except Exception:
            pass
except Exception as e:
    print('Request failed:', e)

STATUS: 201
BODY:
{
  "orderId": 2,
  "restaurantId": 1,
  "customerName": "Alice Presentation",
  "customerPhone": "512-999-0000",
  "totalAmount": 25.48,
  "orderTime": "2025-10-30T04:58:18.550243835",
  "estimatedDeliveryTime": null,
  "orderStatus": "PENDING",
  "assignedDriverId": null,
  "address": null,
  "tips": 3.0,
  "subtotal": 22.48,
  "deliveryTime": null,
  "items": null
}
created_order_id -> 2


## 4) Retrieve the order you just created

In [5]:
# If the previous cell populated created_order_id this will fetch it; otherwise change the id manually
order_id = globals().get('created_order_id') or 1
try:
    r = requests.get(f'{BASE}/orders/{order_id}', timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 200
BODY:
{
  "orderId": 2,
  "restaurantId": 1,
  "customerName": "Alice Presentation",
  "customerPhone": "512-999-0000",
  "totalAmount": 25.48,
  "orderTime": "2025-10-30T04:58:18.550244",
  "estimatedDeliveryTime": null,
  "orderStatus": "PENDING",
  "assignedDriverId": null,
  "address": {
    "addressId": 2,
    "streetAddress": "123 Maple St",
    "city": "Austin",
    "state": "TX",
    "zipCode": "73301"
  },
  "tips": 3.0,
  "subtotal": 22.48,
  "deliveryTime": null,
  "items": [
    {
      "menuItemId": 1,
      "quantity": 2
    }
  ]
}


## 5) Assign a driver to the order

In [6]:
# NOTE: the scaffold controller reuses an OrderItemRequest shape; we use menuItemId as driverId here
order_id = globals().get('created_order_id') or 1
assign_payload = { 'menuItemId': 1 }  # treat as driverId in current scaffold
try:
    r = requests.post(f'{BASE}/orders/{order_id}/assign-driver', json=assign_payload, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 200
BODY:
{
  "orderId": 2,
  "restaurantId": 1,
  "customerName": "Alice Presentation",
  "customerPhone": "512-999-0000",
  "totalAmount": 25.48,
  "orderTime": "2025-10-30T04:58:18.550244",
  "estimatedDeliveryTime": null,
  "orderStatus": "PENDING",
  "assignedDriverId": 1,
  "address": null,
  "tips": 3.0,
  "subtotal": 22.48,
  "deliveryTime": null,
  "items": null
}


## 6) Update order delivery time (mark delivered)

In [7]:
order_id = globals().get('created_order_id') or 1
delivery_time = (datetime.utcnow() + timedelta(minutes=45)).isoformat()
try:
    # This endpoint takes deliveryTime as a query param in the current scaffold
    r = requests.post(f'{BASE}/orders/{order_id}/delivery', params={'deliveryTime': delivery_time}, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 200
BODY:
{
  "orderId": 2,
  "restaurantId": 1,
  "customerName": "Alice Presentation",
  "customerPhone": "512-999-0000",
  "totalAmount": 25.48,
  "orderTime": "2025-10-30T04:58:18.550244",
  "estimatedDeliveryTime": null,
  "orderStatus": "DELIVERED",
  "assignedDriverId": 1,
  "address": null,
  "tips": 3.0,
  "subtotal": 22.48,
  "deliveryTime": "2025-10-30T05:43:49.551565",
  "items": null
}


C:\Users\hiren\AppData\Local\Temp\ipykernel_19752\3158966759.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  delivery_time = (datetime.utcnow() + timedelta(minutes=45)).isoformat()


## 7) Create a new staff account

In [8]:
staff_payload = { 'username': 'presentation_staff', 'password': 'pass123', 'firstname': 'Pres', 'lastname': 'Staff' }
try:
    r = requests.post(f'{BASE}/staff', json=staff_payload, timeout=10)
    print_resp(r)
    if r.status_code in (200,201):
        created = r.json() if r.headers.get('Content-Type','').startswith('application/json') else None
        if created and isinstance(created, dict):
            created_staff_username = created.get('username') or staff_payload['username']
            print('created_staff_username ->', created_staff_username)
except Exception as e:
    print('Request failed:', e)

STATUS: 201
BODY:
{
  "username": "presentation_staff",
  "firstname": "Pres",
  "lastname": "Staff"
}
created_staff_username -> presentation_staff


## 8) Delete / Inactivate staff account

In [10]:
staff_username = globals().get('created_staff_username') or 'presentation_staff'
try:
    r = requests.delete(f'{BASE}/staff/{staff_username}', timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:02:46.465+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/staff/presentation_staff"
}


## 9) Create a new driver record

In [9]:
driver_payload = { 'firstname': 'Demo', 'lastname': 'Driver' }
try:
    r = requests.post(f'{BASE}/drivers', json=driver_payload, timeout=10)
    print_resp(r)
    if r.status_code in (200,201):
        try:
            d = r.json()
            created_driver_id = d.get('driverId') or d.get('id')
            print('created_driver_id ->', created_driver_id)
        except Exception:
            pass
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:02.266+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/drivers"
}


## 10) Delete / Inactivate driver record

In [10]:
driver_id = globals().get('created_driver_id') or 1
try:
    r = requests.delete(f'{BASE}/drivers/{driver_id}', timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:09.524+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/drivers/1"
}


## 11) Restaurant owner successful login

In [11]:
owner_payload = { 'username': 'tacofiesta_admin', 'password': 'password123' }
try:
    r = requests.post(f'{BASE}/restaurant/login', json=owner_payload, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:13.985+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurant/login"
}


## 12) Restaurant owner invalid login

In [12]:
owner_payload = { 'username': 'tacofiesta_admin', 'password': 'wrong' }
try:
    r = requests.post(f'{BASE}/restaurant/login', json=owner_payload, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:18.034+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurant/login"
}


## 13) Restaurant owner modifies a menu item

In [13]:
# Example: update menu item with id=1 for restaurantId=1
menu_update = { 'price': 9.99, 'availability': 'AVAILABLE', 'description': 'Updated for demo' }
try:
    r = requests.put(f'{BASE}/restaurants/1/menu/1', json=menu_update, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:21.282+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurants/1/menu/1"
}


## 14) Restaurant owner modifies operating hours

In [14]:
# Example: update operating hour with id=1 for restaurantId=1
hours_update = { 'weekDay': 'Monday', 'openTime': '09:00:00', 'closeTime': '21:00:00' }
try:
    r = requests.put(f'{BASE}/restaurants/1/hours/1', json=hours_update, timeout=10)
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:24.503+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurants/1/hours/1"
}


## 15) New restaurant registration request

In [15]:
reg_payload = {
  'name': 'Demo Bistro',
  'cuisineType': 'Fusion',
  'pictureUrl': '',
  'addressId': 1,
  'phoneNumber': '555-000-1111',
  'contactPersonName': 'Owner One',
  'emailAddress': 'owner@demobistro.com'
}
try:
    r = requests.post(f'{BASE}/restaurants', json=reg_payload, timeout=10)
    print_resp(r)
    if r.status_code in (200,201):
        try:
            created_restaurant = r.json()
            created_restaurant_id = created_restaurant.get('restaurantId') or created_restaurant.get('restaurant_id')
            print('created_restaurant_id ->', created_restaurant_id)
        except Exception:
            pass
except Exception as e:
    print('Request failed:', e)

STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:26.992+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurants"
}


## 16) Admin approves / rejects a registration

In [16]:
# Replace 2 with the pending restaurant id created earlier if available
pending_id = globals().get('created_restaurant_id') or 2
try:
    r = requests.post(f'{BASE}/admin/restaurants/{pending_id}/approve', timeout=10)
    print('Approve response:')
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

try:
    r2 = requests.post(f'{BASE}/admin/restaurants/{pending_id}/reject', timeout=10)
    print('Reject response:')
    print_resp(r2)
except Exception as e:
    print('Request failed:', e)

Approve response:
STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:30.737+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/admin/restaurants/2/approve"
}
Reject response:
STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:30.757+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/admin/restaurants/2/reject"
}


## 17) Restaurant withdrawal request and admin actions

In [17]:
rest_id = globals().get('created_restaurant_id') or 1
try:
    r = requests.post(f'{BASE}/restaurants/{rest_id}/withdraw', timeout=10)
    print('Withdraw request:')
    print_resp(r)
except Exception as e:
    print('Request failed:', e)

# Admin approve/reject withdrawal
try:
    ra = requests.post(f'{BASE}/admin/restaurants/{rest_id}/withdraw/approve', timeout=10)
    print('Admin approve withdrawal:')
    print_resp(ra)
except Exception as e:
    print('Request failed:', e)

try:
    rr = requests.post(f'{BASE}/admin/restaurants/{rest_id}/withdraw/reject', timeout=10)
    print('Admin reject withdrawal:')
    print_resp(rr)
except Exception as e:
    print('Request failed:', e)

Withdraw request:
STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:41.613+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/restaurants/1/withdraw"
}
Admin approve withdrawal:
STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:41.630+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/admin/restaurants/1/withdraw/approve"
}
Admin reject withdrawal:
STATUS: 404
BODY:
{
  "timestamp": "2025-10-30T04:59:41.645+00:00",
  "status": 404,
  "error": "Not Found",
  "path": "/api/admin/restaurants/1/withdraw/reject"
}


## 18) Interactive runner (runs the main tests and pauses between them)
Run this cell to execute a small subset of tests sequentially; it will prompt you to press Enter between steps.
If you prefer manual control, run the earlier cells individually instead.

In [18]:
def run_demo_sequence():
    tests = [
        ('Staff successful login', lambda: requests.post(f'{BASE}/staff/login', json={'username':'staff_mike','password':'staff123'})),
        ('Create order', lambda: requests.post(f'{BASE}/orders', json={ 'restaurantId':1,'customerName':'Demo','customerPhone':'512-000-0000','address':{'streetAddress':'123','city':'Austin','state':'TX','zipCode':'73301'},'items':[{'menuItemId':1,'quantity':1}],'tips':1.0,'subtotal':5.0})),
        ('Get order (id=1)', lambda: requests.get(f'{BASE}/orders/1')),
    ]
    for name, fn in tests:
        print('
===', name, '===')
        try:
            r = fn()
            print_resp(r)
        except Exception as e:
            print('Request failed:', e)
        input('Press Enter to continue to next test...')

# Uncomment to run interactively from this cell:
# run_demo_sequence()

SyntaxError: unterminated string literal (detected at line 8) (1227763391.py, line 8)

---
## Notes
- Some endpoints used in this notebook may not be implemented yet in the backend; running those cells will show error responses which helps you identify missing controllers or DTO mismatches.
- For presentation, run Docker Compose then open this notebook and proceed cell-by-cell.
- If you want, I can also add pytest-style unit/integration tests that assert expected behavior and can be run headlessly.